In [1]:
import pandas as pd
import numpy
import matplotlib

In [2]:
fertility = pd.read_csv("data/age_specific_fertility_rates.csv")
birth_death = pd.read_csv("data/birth_death_growth_rates.csv") 
country_area = pd.read_csv("data/country_names_area.csv")
midyear_pop_5yrs = pd.read_csv("data/midyear_population_5yr_age_sex.csv")
mortality_life = pd.read_csv("data/mortality_life_expectancy.csv")
midyear_population = pd.read_csv("data/midyear_population.csv")

In [12]:
country_area

,country_code,country_name,country_area
0,AF,Afghanistan,652230.0
1,AL,Albania,27398.0
2,AG,Algeria,2381741.0
3,AQ,American Samoa,199.0
4,AN,Andorra,468.0
...,...,...,...
223,WE,West Bank,5640.0
224,WI,Western Sahara,266000.0
225,YM,Yemen,527968.0
226,ZA,Zambia,743398.0


In [13]:
midyear_pop_5yrs

,country_code,country_name,year,total_flag,starting_age,age_group_indicator,ending_age,midyear_population,midyear_population_male,midyear_population_female
0,NO,Norway,2036,A,0,-,4,324151,166113,158038
1,EG,Egypt,2021,A,100,+,0,327,117,210
2,SN,Singapore,2042,A,90,-,94,102198,40128,62070
3,ZI,Zimbabwe,2005,A,85,-,89,19925,9055,10870
4,SI,Slovenia,2035,A,0,-,4,67365,34744,32621
...,...,...,...,...,...,...,...,...,...,...
333075,DO,Dominica,2005,A,30,-,34,4483,2257,2226
333076,SO,Somalia,1990,A,55,-,59,141082,66376,74706
333077,BT,Bhutan,2024,A,80,-,84,7094,3576,3518
333078,NS,Suriname,2021,A,60,-,64,22898,11210,11688


In [14]:
midyear_population

,country_code,country_name,year,yearly_midyear_pop
0,SI,Slovenia,2013,1992690
1,SI,Slovenia,2025,1907560
2,SI,Slovenia,2024,1917093
3,SI,Slovenia,2023,1926279
4,SI,Slovenia,2022,1935080
...,...,...,...,...
23023,BP,Solomon Islands,1980,230768
23024,BP,Solomon Islands,1979,222917
23025,BP,Solomon Islands,1978,215085
23026,BP,Solomon Islands,1977,207278


In [15]:
#replacing column name 
midyear_population.rename(columns={'midyear_population': 'yearly_midyear_pop'}, inplace=True)

# merge country_area w/ midyear_pop
population_data = pd.merge(midyear_population, country_area, on=["country_code", "country_name"], how="left")
population_data


In [17]:
# merge w/ midyear 5 yrs
population_data_2 = pd.merge(population_data, midyear_pop_5yrs, on=["country_code", "country_name", "year"], how="left")
population_data_2

,country_code,country_name,year,yearly_midyear_pop,country_area,total_flag,starting_age,age_group_indicator,ending_age,midyear_population,midyear_population_male,midyear_population_female
0,SI,Slovenia,2013,1992690,20151.0,A,15.0,-,19.0,94879.0,48526.0,46353.0
1,SI,Slovenia,2013,1992690,20151.0,A,40.0,-,44.0,143423.0,72399.0,71024.0
2,SI,Slovenia,2013,1992690,20151.0,A,80.0,-,84.0,51422.0,17217.0,34205.0
3,SI,Slovenia,2013,1992690,20151.0,A,45.0,-,49.0,152663.0,75513.0,77150.0
4,SI,Slovenia,2013,1992690,20151.0,A,100.0,+,0.0,397.0,67.0,330.0
...,...,...,...,...,...,...,...,...,...,...,...,...
340963,BP,Solomon Islands,1977,207278,27986.0,A,35.0,-,39.0,10043.0,5254.0,4789.0
340964,BP,Solomon Islands,1977,207278,27986.0,A,90.0,-,94.0,62.0,38.0,24.0
340965,BP,Solomon Islands,1977,207278,27986.0,A,65.0,-,69.0,2417.0,1508.0,909.0
340966,BP,Solomon Islands,1977,207278,27986.0,A,100.0,+,0.0,1.0,0.0,1.0


In [21]:
age_bins = [0,20,40,60,80,100]
age_labels = ['0-20', '20-40', '40-60', '60-80' ,'80-100+']

#adding new column for groups
population_data_2['age_group'] = pd.cut(population_data_2['ending_age'], bins=age_bins, labels=age_labels, right=False)

aggregated_data = population_data_2.groupby(['country_code', 'country_name', 'year', 'age_group']).agg({
    'midyear_population': 'sum',
    'midyear_population_male': 'sum',
    'midyear_population_female': 'sum',
    'country_area': 'first'
}).reset_index()


KeyboardInterrupt: 

In [ ]:
aggregated_data

In [9]:
final_population_data = pd.merge(aggregated_data, population_data[['country_code', 'country_name', 'year', 'yearly_midyear_pop', 'country_area']].drop_duplicates(), on=['country_code', 'country_name', 'year'], how='left')



In [10]:
final_population_data

,country_code,country_name,year,age_group,midyear_population,midyear_population_male,midyear_population_female,yearly_midyear_pop,country_area
0,AA,Afghanistan,1950,0-20,0.0,0.0,0.0,NaN,NaN
1,AA,Afghanistan,1950,20-40,0.0,0.0,0.0,NaN,NaN
2,AA,Afghanistan,1950,40-60,0.0,0.0,0.0,NaN,NaN
3,AA,Afghanistan,1950,60-80,0.0,0.0,0.0,NaN,NaN
4,AA,Afghanistan,1950,80-100+,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
26251915,ZI,Zimbabwe,2050,0-20,35202756.0,17949950.0,17252806.0,25198196.0,386847.0
26251916,ZI,Zimbabwe,2050,20-40,8074773.0,4159035.0,3915738.0,25198196.0,386847.0
26251917,ZI,Zimbabwe,2050,40-60,4478959.0,2385429.0,2093530.0,25198196.0,386847.0
26251918,ZI,Zimbabwe,2050,60-80,2348873.0,1168522.0,1180351.0,25198196.0,386847.0
